In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

# Try SciPy for stable Φ/Φ^{-1}; fall back to numpy/erf if not present.
try:
    from scipy.stats import norm
    def phi(z): return norm.cdf(z)
    def phi_inv(u): return norm.ppf(np.clip(u, 1e-12, 1-1e-12))
    HAS_SCIPY = True
except Exception:
    from math import erf, sqrt
    def phi(z): return 0.5 * (1.0 + erf(z / sqrt(2.0)))
    # Simple, decent inverse via erfinv if available
    try:
        from scipy.special import erfinv
        def phi_inv(u):
            u = np.clip(u, 1e-12, 1-1e-12)
            return sqrt(2.0)*erfinv(2*u - 1)
    except Exception:
        raise ImportError("This notebook prefers SciPy. Please `pip install scipy` for norm.ppf.")

rng = np.random.default_rng(12345)

# Paths (use your actual export locations)
CONT_PATH = "data/input/simple_params_1030/cs_2019_continuous_by_nace2.csv"
CORR_PATH = "data/input/simple_params_1030/cs_2019_corr_long_by_nace2.csv"

# How many firms to simulate per NACE2?
# We'll match the max 'N' reported across continuous vars for that NACE2 (below).
SIM_SCALE = 1.0  # 1.0 = match counts; e.g. 0.5 = half, 2.0 = double


In [2]:
cont = pd.read_csv(CONT_PATH)
corr_long = pd.read_csv(CORR_PATH)

# Clean column names we’ll use
cont.columns = [c.strip() for c in cont.columns]
corr_long.columns = [c.strip() for c in corr_long.columns]

# Percentile columns available in your export
pct_cols = ["p0001","p01","p05","p10","p25","p50","p75","p90","p95","p99","p999"]
base_cols = ["nace2","var","N","min","max","median","mean","sd","skewness","excess_kurtosis"]
assert all(pc in cont.columns for pc in pct_cols), "Some percentile columns missing."

# Build a quick lookup of group sizes per NACE2
N_by_nace = cont.groupby("nace2")["N"].max().astype(int)
nace_list = list(N_by_nace.index)

# Continuous variables present (union over all groups)
all_vars = cont["var"].unique().tolist()


In [3]:
# For each (nace2, var) we’ll store U-grid and X-grid in dicts
inv_grids = {}  # key: (nace2, var) → dict with 'u' and 'x' numpy arrays

# Probability grid that matches the columns we have, plus endpoints
u_grid = np.array([0.0,   0.001, 0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99, 0.999, 1.0])

for g in nace_list:
    sub = cont[cont["nace2"] == g]
    for v in sub["var"].unique():
        row = sub[sub["var"] == v].iloc[0]
        # Build X-grid in the same order as u_grid
        x_vals = [
            row["min"],
            row["p0001"], row["p01"], row["p05"], row["p10"], row["p25"],
            row["p50"], row["p75"], row["p90"], row["p95"], row["p99"], row["p999"],
            row["max"],
        ]
        x_grid = np.asarray(x_vals, dtype=float)

        # Ensure monotonicity (just in case of minor rounding noise)
        x_grid = np.maximum.accumulate(x_grid)

        inv_grids[(g, v)] = {"u": u_grid, "x": x_grid}


In [4]:
def build_psd_corr_for_group(g, var_list):
    # Take rows for this nace2
    g_corr = corr_long[corr_long["nace2"] == g].copy()
    # Ensure we only include requested var_list
    g_corr = g_corr[g_corr["var1"].isin(var_list) & g_corr["var2"].isin(var_list)]
    # Wide matrix
    corr = pd.pivot_table(g_corr, index="var1", columns="var2", values="cor")
    # Reindex to full var_list on both axes
    corr = corr.reindex(index=var_list, columns=var_list)
    # Fill diag with 1, off-diag missing with 0
    np.fill_diagonal(corr.values, 1.0)
    corr = corr.fillna(0.0)

    # Symmetrize
    M = corr.values
    M = 0.5 * (M + M.T)

    # Eigenvalue clipping to ensure PSD
    w, V = np.linalg.eigh(M)
    w_clipped = np.clip(w, 1e-8, None)
    M_psd = (V * w_clipped) @ V.T

    # Normalize back to correlation (set diag = 1 and rescale slight drift)
    d = np.sqrt(np.diag(M_psd))
    M_psd = M_psd / np.outer(d, d)
    np.fill_diagonal(M_psd, 1.0)

    return pd.DataFrame(M_psd, index=var_list, columns=var_list)


In [5]:
synthetic_blocks = []

for g in nace_list:
    # Variables with a marginal (grid) in this group:
    vars_marg = cont.loc[cont["nace2"] == g, "var"].unique().tolist()

    # Variables present in correlation for this group:
    g_vars_corr = pd.unique(
        corr_long.loc[corr_long["nace2"] == g, ["var1","var2"]].values.ravel("K")
    ).tolist()

    # Intersection = variables we'll simulate for this group
    var_list = [v for v in all_vars if v in vars_marg and v in g_vars_corr]
    if len(var_list) == 0:
        continue

    # Correlation matrix (PSD)
    C = build_psd_corr_for_group(g, var_list).values

    # Sample size for the group
    N_sim = int(max(1, round(N_by_nace[g] * SIM_SCALE)))

    # Draw correlated normals via eigen or cholesky
    # Cholesky can occasionally fail if near-singular; use eigh instead
    w, V = np.linalg.eigh(C)
    w = np.clip(w, 0, None)
    A = V @ np.diag(np.sqrt(w)) @ V.T

    Z = rng.standard_normal(size=(N_sim, len(var_list)))
    Z = Z @ A.T

    # Map to uniforms
    U = phi(Z)

    # Map uniforms to each variable via its (nace2,var) inverse CDF grid
    data = {}
    for j, v in enumerate(var_list):
        grid = inv_grids[(g, v)]
        # linear interpolation on the grid
        x_sim = np.interp(U[:, j], grid["u"], grid["x"])
        data[v] = x_sim

    block = pd.DataFrame(data)
    block.insert(0, "nace2", g)
    block.insert(1, "row_id", np.arange(N_sim, dtype=int))
    synthetic_blocks.append(block)

synthetic = pd.concat(synthetic_blocks, ignore_index=True)
synthetic.head()


,nace2,row_id,age,tanass_clean,eszk,sales_clean,pretax,persexp_clean,satok,export_value,ereduzem,emp,liabilities
0,10,0,0.006364,66772.594760,29923.832264,18529.912941,-37.104270,94.336264,-9188.041831,1.180157e+06,438.422491,0.000000,191021.758624
1,10,1,25.299281,10151.716078,78304.886780,158273.631374,12682.838454,86948.076892,8450.160706,3.265402e+06,22435.187059,40.529525,37910.896832
2,10,2,25.063239,151.088562,113422.626325,786718.913072,4273.047811,60718.671378,737127.712029,4.545077e+07,6657.828324,21.728193,10879.315308
3,10,3,1.469206,107.778810,27571.399007,69855.839155,-187.071126,6329.708135,18738.391276,5.996941e+05,-74.706535,8.954354,11129.220485
4,10,4,10.689950,5465.810743,250913.386580,879128.632820,8007.565266,66960.188081,152867.314478,7.389195e+06,15639.383140,24.826914,98656.285291


print("Synthetic shape:", synthetic.shape)
print("Per NACE2 counts:")
print(synthetic["nace2"].value_counts().sort_index())

# Save
Path("data/synthetic").mkdir(parents=True, exist_ok=True)
# synthetic.to_parquet("data/synthetic/sim_cs2019_by_nace2_gausscop.parquet", index=False)
# synthetic.to_csv("data/synthetic/sim_cs2019_by_nace2_gausscop.csv", index=False)

# Optional: a tiny sanity peek at means vs targets for one group/var
g0 = synthetic["nace2"].unique()[0]
v0 = synthetic.columns[synthetic.columns.get_loc("row_id")+1]  # first simulated var
print(f"\nExample sanity — group {g0}, var {v0}")
print("simulated mean:", synthetic.loc[synthetic["nace2"]==g0, v0].mean())
target_mean = cont.query("nace2 == @g0 and var == @v0")["mean"].iloc[0]
print("target mean   :", target_mean)


In [6]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(42)

dummy_path = "data/input/simple_params_1030/cs_2019_dummy_shares_by_nace2.csv"
reg_path   = "data/input/simple_params_1030/cs_2019_category_shares_by_nace2_region.csv"
cnty_path  = "data/input/simple_params_1030/cs_2019_category_shares_by_nace2_county.csv"
own_path   = "data/input/simple_params_1030/cs_2019_category_shares_by_nace2_owner.csv"

dshare = pd.read_csv(dummy_path)
reg_sh = pd.read_csv(reg_path)   if Path(reg_path).exists()  else pd.DataFrame(columns=["nace2","region","N","share"])
cnty_sh= pd.read_csv(cnty_path)  if Path(cnty_path).exists() else pd.DataFrame(columns=["nace2","county","N","share"])
own_sh = pd.read_csv(own_path)   if Path(own_path).exists()  else pd.DataFrame(columns=["nace2","firm_owner","N","share"])

# Keep only what we need
dshare = dshare[["nace2","share_export","share_grant","share_exit"]].copy()

# Make sure shares are within [0,1]
for c in ["share_export","share_grant","share_exit"]:
    if c in dshare.columns:
        dshare[c] = dshare[c].clip(0, 1)


In [7]:
syn = synthetic.copy()  # from your previous cells
syn["has_export"] = np.nan
syn["has_grant"]  = np.nan
syn["exit"]       = np.nan

for g, gdf in syn.groupby("nace2"):
    idx = gdf.index
    row = dshare[dshare["nace2"] == g]
    if row.empty:
        # no shares for this group; skip
        continue
    p_export = float(row["share_export"].iloc[0]) if "share_export" in row else 0.0
    p_grant  = float(row["share_grant"].iloc[0])  if "share_grant"  in row else 0.0
    p_exit   = float(row["share_exit"].iloc[0])   if "share_exit"   in row else 0.0

    syn.loc[idx, "has_export"] = rng.binomial(1, p_export, size=len(idx))
    syn.loc[idx, "has_grant"]  = rng.binomial(1, p_grant,  size=len(idx))
    syn.loc[idx, "exit"]       = rng.binomial(1, p_exit,   size=len(idx))

# Cast to integers
syn["has_export"] = syn["has_export"].astype("Int64")
syn["has_grant"]  = syn["has_grant"].astype("Int64")
syn["exit"]       = syn["exit"].astype("Int64")


In [8]:
# Prepare containers
syn["region"]     = pd.NA
syn["county"]     = pd.NA
syn["firm_owner"] = pd.NA

# Region
if not reg_sh.empty:
    reg_sh = reg_sh.rename(columns={reg_sh.columns[0]: "region"} if reg_sh.columns[0] != "region" else {})
    # keep columns: nace2, region, share
    keep_cols = [c for c in ["nace2","region","share"] if c in reg_sh.columns]
    reg_sh = reg_sh[keep_cols].dropna(subset=["region"]).copy()
    # normalize shares per group (just in case)
    reg_sh["share"] = reg_sh["share"].clip(lower=0)
    reg_sh["share"] = reg_sh.groupby("nace2")["share"].transform(lambda s: s / s.sum() if s.sum() > 0 else s)

    for g, gdf in syn.groupby("nace2"):
        idx = gdf.index
        opts = reg_sh[reg_sh["nace2"] == g]
        if opts.empty:
            continue
        cats = opts["region"].astype(str).tolist()
        probs= opts["share"].astype(float).to_numpy()
        # guard for rounding: re-normalize
        probs = probs / probs.sum() if probs.sum() > 0 else np.ones_like(probs)/len(probs)
        draws = rng.choice(cats, size=len(idx), p=probs, replace=True)
        syn.loc[idx, "region"] = draws

# County
if not cnty_sh.empty:
    cnty_sh = cnty_sh.rename(columns={cnty_sh.columns[0]: "county"} if cnty_sh.columns[0] != "county" else {})
    keep_cols = [c for c in ["nace2","county","share"] if c in cnty_sh.columns]
    cnty_sh = cnty_sh[keep_cols].dropna(subset=["county"]).copy()
    cnty_sh["share"] = cnty_sh["share"].clip(lower=0)
    cnty_sh["share"] = cnty_sh.groupby("nace2")["share"].transform(lambda s: s / s.sum() if s.sum() > 0 else s)

    for g, gdf in syn.groupby("nace2"):
        idx = gdf.index
        opts = cnty_sh[cnty_sh["nace2"] == g]
        if opts.empty:
            continue
        cats = opts["county"].astype(str).tolist()
        probs= opts["share"].astype(float).to_numpy()
        probs = probs / probs.sum() if probs.sum() > 0 else np.ones_like(probs)/len(probs)
        draws = rng.choice(cats, size=len(idx), p=probs, replace=True)
        syn.loc[idx, "county"] = draws

# Firm owner
if not own_sh.empty:
    own_sh = own_sh.rename(columns={own_sh.columns[0]: "firm_owner"} if own_sh.columns[0] != "firm_owner" else {})
    keep_cols = [c for c in ["nace2","firm_owner","share"] if c in own_sh.columns]
    own_sh = own_sh[keep_cols].dropna(subset=["firm_owner"]).copy()
    own_sh["share"] = own_sh["share"].clip(lower=0)
    own_sh["share"] = own_sh.groupby("nace2")["share"].transform(lambda s: s / s.sum() if s.sum() > 0 else s)

    for g, gdf in syn.groupby("nace2"):
        idx = gdf.index
        opts = own_sh[own_sh["nace2"] == g]
        if opts.empty:
            continue
        cats = opts["firm_owner"].astype(str).tolist()
        probs= opts["share"].astype(float).to_numpy()
        probs = probs / probs.sum() if probs.sum() > 0 else np.ones_like(probs)/len(probs)
        draws = rng.choice(cats, size=len(idx), p=probs, replace=True)
        syn.loc[idx, "firm_owner"] = draws

for col in ["emp", "age"]:
    if col in syn.columns:
        vals = pd.to_numeric(syn[col], errors="coerce")          # ensure numeric
        vals = np.rint(vals)                                     # round to nearest int
        vals = np.clip(vals, 0, None)                            # no negatives
        syn[col] = pd.Series(vals).astype("Int64")  


In [9]:
lab = pd.read_excel("data/nace2_labels.xlsx")
lab["nace2"] = lab["nace2"].apply(str)
lab["nace2"] = lab["nace2"].str.extract(r"(\d+)", expand=False).fillna("").str.zfill(2).str[:2]

syn = syn.merge(lab, on="nace2", how="left")
syn["nace2_name_code"] = syn["name_hu"].fillna("NACE " + syn["nace2"]) + " (" + syn["nace2"] + ")"


In [10]:
print("Synthetic with dummies + categoricals:", syn.shape)
print(syn.dtypes.head(12))

# Quick share checks (simulated vs target) for one group
g0 = syn["nace2"].unique()[0]
print("\nExample shares — group", g0)
sim_row = syn[syn["nace2"]==g0]
print("export", sim_row["has_export"].mean(), 
      "grant", sim_row["has_grant"].mean(), 
      "exit",  sim_row["exit"].mean())

# Save
Path("data/synthetic").mkdir(parents=True, exist_ok=True)
syn.to_parquet("data/synthetic/sim_cs2019_by_nace2_withcats.parquet", index=False)
# syn.to_csv("data/synthetic/sim_cs2019_by_nace2_withcats.csv", index=False)


Synthetic with dummies + categoricals: (829836, 21)
nace2             object
row_id             int64
age                Int64
tanass_clean     float64
eszk             float64
sales_clean      float64
pretax           float64
persexp_clean    float64
satok            float64
export_value     float64
ereduzem         float64
emp                Int64
dtype: object

Example shares — group 10
export 0.06861198738170347 grant 0.03680336487907466 exit 0.11277602523659307


In [11]:
syn["nace2"].value_counts()

nace2
ALL    414918
47      44539
68      36993
46      30452
43      21576
        ...  
98          6
34          3
76          2
57          1
67          1
Name: count, Length: 85, dtype: int64

In [12]:
syn["nace2_name_code"].value_counts()

nace2_name_code
NACE ALL (ALL)                                                         414918
Kiskereskedelem (kivéve: gépjármű, motorkerékpár) (47)                  44539
INGATLANÜGYLETEK (68)                                                   36993
Nagykereskedelem (kivéve: jármű, motorkerékpár) (46)                    30452
Speciális szaképítés (43)                                               21576
                                                                        ...  
Háztartás termék-előállítása, szolgáltatása saját fogyasztásra (98)         6
NACE 34 (34)                                                                3
NACE 76 (76)                                                                2
NACE 57 (57)                                                                1
NACE 67 (67)                                                                1
Name: count, Length: 85, dtype: int64